In [102]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go
from streamlit_lottie import st_lottie
import json
import numpy  as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def load_lottiefile(filepath: str):
    with open(filepath, "r") as f:
        return json.load(f)


from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv("Titles.csv")
df= df[df['type'] == 'MOVIE']
credits_df = pd.read_csv("Credits.csv")

# df["director"] = pd.merge(
#     df, credits_df[credits_df["role"] == "DIRECTOR"], on="id", how="left"
# )["name"].replace(np.nan, None)

# df["actors"] = pd.merge(
#     df,
#     pd.merge(df, credits_df[credits_df["role"] == "ACTOR"], on="id", how="left")
#     .groupby("id")["name"]
#     .apply(lambda x: x.tolist() if x is not np.nan else None),
#     on="id",
#     how="left",
# )["name"].apply(lambda x: ["" if i is np.nan else str(i) for i in x])

# df["actors"] = df["actors"].replace(np.nan, "")

df["overview"] = (
    (
        + df["description"].astype(str)
        + " "
        + df["genres"].apply(lambda x: " ".join(x))
    )
    .str.lower()
    .str.replace("\n", " ")
    .str.replace("—", "")
)


In [103]:

count = CountVectorizer(stop_words="english", ngram_range=(1, 5))
count_matrix = count.fit_transform(df["overview"])
cosine_sim = cosine_similarity(count_matrix, count_matrix)
indices = pd.Series(df.index, index=df["title"])
count = CountVectorizer(stop_words="english", ngram_range=(1, 5))
count_matrix = count.fit_transform(df["overview"])
cosine_sim = cosine_similarity(count_matrix, count_matrix)
indices = pd.Series(df.index, index=df["title"])

In [104]:
df.to_parquet('data.parquet')

In [105]:
cosine_sim = pd.DataFrame(cosine_sim)
cosine_sim.columns = cosine_sim.columns.astype(str)
cosine_sim.to_parquet('cosine_sim.parquet', compression='gzip')

In [106]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go
from streamlit_lottie import st_lottie
import json
import numpy  as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [107]:
df = pd.read_parquet('data.parquet')
cosine_sim = pd.read_parquet('cosine_sim.parquet').to_numpy()

In [108]:
indices = pd.Series(df.index, index=df["title"])

In [109]:
movie_list = df['title'].unique()

In [110]:

def get_recommendations(title, cosine_sim, top_k=5):
    
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[0:20]
    movie_indices = [i[0] for i in sim_scores if i[0] != idx]

    return (
        df.iloc[movie_indices]
        .sort_values(["imdb_votes", "imdb_score"], ascending=False)[
            ["title", "description", "genres", "imdb_score"]
        ]
        .reset_index(drop=True)
        .head(top_k)
    )


In [111]:
movie_list

array(['Taxi Driver', 'Monty Python and the Holy Grail', 'Life of Brian',
       ..., 'Edis Starlight', 'Clash', 'Shadow Parties'], dtype=object)

In [118]:
recommended_movie_names = get_recommendations(movie_list[3], cosine_sim=cosine_sim, top_k=10)    

In [130]:
def fetch_poster(movie_id):
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key=020b311fe0559698373a16008dc6a672&language=en-US'.format(movie_id))
    data = response.json()
    return "https://image.tmdb.org/t/p/w500/" + data['poster_path']

In [92]:
import streamlit as st
import requests
import pandas as pd

# st.set_page_config(
#     page_title="Movie recommender system",
#     page_icon="🕸",
#     layout="wide",
#     initial_sidebar_state="expanded",
#     menu_items={
#         'Get Help': 'https://github.com/santanukumar666/Movie-Recommendations-system',
#         'About': "## A content based movie recommendation system build using cosine similarity"
#     }
# )

# st.header('Movie Recommender System')


def fetch_poster(movie_id):
    url = "https://api.themoviedb.org/3/movie/{}?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US".format(
        movie_id)
    data = requests.get(url)
    data = data.json()
    poster_path = data['poster_path']
    full_path = "https://image.tmdb.org/t/p/w500/" + poster_path
    return full_path


In [94]:
df

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,director,actors,overview
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,48,['documentation'],['US'],1.0,None,NaN,NaN,0.600,NaN,None,[],this collection includes 12 world war ii-era p...
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,113,"['crime', 'drama']",['US'],NaN,tt0075314,8.3,795222.0,27.612,8.2,Martin Scorsese,"[Robert De Niro, Jodie Foster, Albert Brooks, ...",a mentally unstable vietnam war veteran works ...
2,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['comedy', 'fantasy']",['GB'],NaN,tt0071853,8.2,530877.0,18.216,7.8,Terry Jones,"[Graham Chapman, John Cleese, Eric Idle, Terry...","king arthur, accompanied by his squire, recrui..."
3,tm70993,Life of Brian,MOVIE,"Brian Cohen is an average young Jewish man, bu...",1979,R,94,['comedy'],['GB'],NaN,tt0079470,8.0,392419.0,17.505,7.8,Terry Gilliam,"[Graham Chapman, John Cleese, Terry Gilliam, E...","brian cohen is an average young jewish man, bu..."
4,tm190788,The Exorcist,MOVIE,12-year-old Regan MacNeil begins to adapt an e...,1973,R,133,['horror'],['US'],NaN,tt0070047,8.1,391942.0,95.337,7.7,Terry Jones,"[Ellen Burstyn, Linda Blair, Max von Sydow, Le...",12-year-old regan macneil begins to adapt an e...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5801,tm1014599,Fine Wine,MOVIE,A beautiful love story that can happen between...,2021,None,100,"['romance', 'drama']",['NG'],NaN,tt13857480,6.9,39.0,0.966,NaN,Renée Godfrey,"[Richard Mofe-Damijo, Ego Nwosu, Keppy Ekpenyo...",a beautiful love story that can happen between...
5802,tm1108171,Edis Starlight,MOVIE,Rising star Edis's career journey with ups and...,2021,None,74,"['music', 'documentation']",[],NaN,None,NaN,NaN,1.036,8.5,David Charhon,[Edis Görgülü],rising star edis's career journey with ups and...
5803,tm1045018,Clash,MOVIE,A man from Nigeria returns to his family in Ca...,2021,None,88,"['family', 'drama']","['NG', 'CA']",NaN,tt14620732,6.5,32.0,0.709,NaN,Jung Jee-hyun,"[Omoni Oboli, Stephanie Okereke-Linus, Merlisa...",a man from nigeria returns to his family in ca...
5804,tm1098060,Shadow Parties,MOVIE,A family faces destruction in a long-running c...,2021,None,116,"['action', 'thriller']",[],NaN,tt10168094,6.2,9.0,2.186,NaN,Jon Clay,"[Jide Kosoko, Omotola Jalade-Ekeinde, Yemi Bla...",a family faces destruction in a long-running c...
